In [29]:
!pip install exifread geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.6 MB/s eta 0:00:00


In [30]:
import os
import exifread
import csv
from geopy.distance import geodesic

def extract_gps_from_image(image_path):
    """Extract GPS coordinates from image metadata."""
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    try:
        lat_ref = tags["GPS GPSLatitudeRef"].values
        lon_ref = tags["GPS GPSLongitudeRef"].values
        lat = tags["GPS GPSLatitude"].values
        lon = tags["GPS GPSLongitude"].values

        def convert_to_degrees(gps_coord):
            d, m, s = [float(x.num) / float(x.den) for x in gps_coord]
            return d + (m / 60.0) + (s / 3600.0)

        latitude = convert_to_degrees(lat)
        longitude = convert_to_degrees(lon)

        if lat_ref != "N":
            latitude = -latitude
        if lon_ref != "E":
            longitude = -longitude

        return (latitude, longitude)


    except KeyError:
        return None

def estimate_overlapping_images(image_dir, distance_threshold_m=10):
    """Estimate which image pairs overlap based on GPS distance."""
    image_files = sorted([
        os.path.join(image_dir, f) for f in os.listdir(image_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

    gps_data = {}
    for img in image_files:
        gps = extract_gps_from_image(img)
        if gps:
            gps_data[img] = gps

    # Compare each image with others
    overlaps = []
    img_list = list(gps_data.items())

    for i in range(len(img_list)):
        img1, coord1 = img_list[i]
        for j in range(i + 1, len(img_list)):
            img2, coord2 = img_list[j]
            dist = geodesic(coord1, coord2).meters
            if dist < distance_threshold_m:
                overlaps.append((img1, img2, dist))

    return overlaps

#  usage
image_folder = "/content/drive/MyDrive/Gregg1_2"
output_file = "/content/overlapping_images.csv"
#overlap_pairs = estimate_overlapping_images(image_folder)
overlap_pairs = estimate_overlapping_images(image_folder, distance_threshold_m=20)


# Display estimated overlapping image pairs
for img1, img2, dist in overlap_pairs:
    print(f"Overlap: {os.path.basename(img1)} <--> {os.path.basename(img2)} | Distance: {dist:.2f} m")

# Save output to CSV
with open(output_file, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Image 1", "Image 2", "Distance (meters)"])
    for img1, img2, dist in overlap_pairs:
        writer.writerow([img1, img2, f"{dist:.2f}"])

print(f"Saved overlapping image pairs to: {output_file}")

Overlap: DJI_0002.JPG <--> DJI_0003.JPG | Distance: 19.16 m
Overlap: DJI_0004.JPG <--> DJI_0005.JPG | Distance: 19.94 m
Overlap: DJI_0010.JPG <--> DJI_0011.JPG | Distance: 19.05 m
Overlap: DJI_0013.JPG <--> DJI_0014.JPG | Distance: 19.26 m
Overlap: DJI_0016.JPG <--> DJI_0017.JPG | Distance: 18.97 m
Overlap: DJI_0020.JPG <--> DJI_0021.JPG | Distance: 19.08 m
Overlap: DJI_0022.JPG <--> DJI_0023.JPG | Distance: 19.10 m
Overlap: DJI_0024.JPG <--> DJI_0025.JPG | Distance: 19.17 m
Overlap: DJI_0025.JPG <--> DJI_0026.JPG | Distance: 19.10 m
Overlap: DJI_0027.JPG <--> DJI_0028.JPG | Distance: 19.00 m
Overlap: DJI_0030.JPG <--> DJI_0031.JPG | Distance: 19.03 m
Overlap: DJI_0033.JPG <--> DJI_0034.JPG | Distance: 19.21 m
Overlap: DJI_0035.JPG <--> DJI_0036.JPG | Distance: 19.10 m
Overlap: DJI_0044.JPG <--> DJI_0045.JPG | Distance: 19.12 m
Overlap: DJI_0046.JPG <--> DJI_0047.JPG | Distance: 18.93 m
Overlap: DJI_0049.JPG <--> DJI_0050.JPG | Distance: 19.16 m
Overlap: DJI_0052.JPG <--> DJI_0053.JPG 

In [31]:
import os
import exifread
from geopy.distance import geodesic

# === Step 1: Extract GPS coordinates from a single image ===
def extract_gps_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    try:
        lat_ref = tags["GPS GPSLatitudeRef"].values
        lon_ref = tags["GPS GPSLongitudeRef"].values
        lat = tags["GPS GPSLatitude"].values
        lon = tags["GPS GPSLongitude"].values
    except KeyError:
        return None  # GPS info missing

    def convert_to_degrees(values):
        d = float(values[0].num) / float(values[0].den)
        m = float(values[1].num) / float(values[1].den)
        s = float(values[2].num) / float(values[2].den)
        return d + (m / 60.0) + (s / 3600.0)

    lat_deg = convert_to_degrees(lat)
    lon_deg = convert_to_degrees(lon)
    if lat_ref != "N":
        lat_deg *= -1
    if lon_ref != "E":
        lon_deg *= -1

    return (lat_deg, lon_deg)

# === Step 2: Build a dictionary of frame => GPS ===
def build_gps_dict(image_folder):
    gps_dict = {}
    for fname in sorted(os.listdir(image_folder)):
        if fname.lower().endswith(".jpg") or fname.lower().endswith(".jpeg") or fname.lower().endswith(".png"):
            full_path = os.path.join(image_folder, fname)
            gps = extract_gps_from_image(full_path)
            if gps:
                gps_dict[fname] = gps
    return gps_dict

# === Step 3: Calculate overlap score based on GPS distance ===
def calculate_overlap_scores(gps_dict, threshold_meters=30):
    overlaps = []
    frames = list(gps_dict.keys())

    for i in range(len(frames) - 1):
        f1, f2 = frames[i], frames[i + 1]
        coord1, coord2 = gps_dict[f1], gps_dict[f2]
        distance = geodesic(coord1, coord2).meters

        if distance > threshold_meters:
            score = 0.0
        else:
            score = 1.0 - (distance / threshold_meters)  # closer = higher score

        overlaps.append((f1, f2, round(score, 3)))

    return overlaps

# === Step 4: Run the pipeline ===
image_folder = "/content/drive/MyDrive/Gregg1_2"
gps_dict = build_gps_dict(image_folder)
overlaps = calculate_overlap_scores(gps_dict)

# === Step 5: Print result ===
print("📍 Frame Overlap Scores:")
for f1, f2, score in overlaps:
    print(f"{f1} <-> {f2} => Overlap Score: {score}")


📍 Frame Overlap Scores:
DJI_0001.JPG <-> DJI_0002.JPG => Overlap Score: 0.0
DJI_0002.JPG <-> DJI_0003.JPG => Overlap Score: 0.361
DJI_0003.JPG <-> DJI_0004.JPG => Overlap Score: 0.015
DJI_0004.JPG <-> DJI_0005.JPG => Overlap Score: 0.335
DJI_0005.JPG <-> DJI_0006.JPG => Overlap Score: 0.0
DJI_0006.JPG <-> DJI_0007.JPG => Overlap Score: 0.329
DJI_0007.JPG <-> DJI_0008.JPG => Overlap Score: 0.054
DJI_0008.JPG <-> DJI_0009.JPG => Overlap Score: 0.333
DJI_0009.JPG <-> DJI_0010.JPG => Overlap Score: 0.0
DJI_0010.JPG <-> DJI_0011.JPG => Overlap Score: 0.365
DJI_0011.JPG <-> DJI_0012.JPG => Overlap Score: 0.05
DJI_0012.JPG <-> DJI_0013.JPG => Overlap Score: 0.04
DJI_0013.JPG <-> DJI_0014.JPG => Overlap Score: 0.358
DJI_0014.JPG <-> DJI_0015.JPG => Overlap Score: 0.048
DJI_0015.JPG <-> DJI_0016.JPG => Overlap Score: 0.046
DJI_0016.JPG <-> DJI_0017.JPG => Overlap Score: 0.368
DJI_0017.JPG <-> DJI_0018.JPG => Overlap Score: 0.0
DJI_0018.JPG <-> DJI_0019.JPG => Overlap Score: 0.03
DJI_0019.JPG <-

In [34]:
import os
import exifread

def extract_gps_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    try:
        # Latitude
        lat_ref = tags["GPS GPSLatitudeRef"].values
        lat = tags["GPS GPSLatitude"].values
        lat_deg = float(lat[0].num) / lat[0].den
        lat_min = float(lat[1].num) / lat[1].den
        lat_sec = float(lat[2].num) / lat[2].den
        latitude = lat_deg + lat_min/60 + lat_sec/3600
        if lat_ref != "N":
            latitude = -latitude

        # Longitude
        lon_ref = tags["GPS GPSLongitudeRef"].values
        lon = tags["GPS GPSLongitude"].values
        lon_deg = float(lon[0].num) / lon[0].den
        lon_min = float(lon[1].num) / lon[1].den
        lon_sec = float(lon[2].num) / lon[2].den
        longitude = lon_deg + lon_min/60 + lon_sec/3600
        if lon_ref != "E":
            longitude = -longitude

        return latitude, longitude

    except KeyError:
        return None  # No GPS data

# Folder containing frames (images)
folder_path = "/content/drive/MyDrive/Gregg1_2"
image_files = sorted(os.listdir(folder_path))

# Extract info
for img in image_files:
    img_path = os.path.join(folder_path, img)
    gps = extract_gps_from_image(img_path)
    print(f"{img} → GPS: {gps}")


DJI_0001.JPG → GPS: (38.98769411111111, -107.88787850000001)
DJI_0002.JPG → GPS: (38.98742111111111, -107.88788069444445)
DJI_0003.JPG → GPS: (38.9872485, -107.88788261111112)
DJI_0004.JPG → GPS: (38.98698230555556, -107.88788500000001)
DJI_0005.JPG → GPS: (38.98680275, -107.88788819444446)
DJI_0006.JPG → GPS: (38.986519694444446, -107.88788794444446)
DJI_0007.JPG → GPS: (38.986338333333336, -107.88788927777779)
DJI_0008.JPG → GPS: (38.98608275, -107.88789302777778)
DJI_0009.JPG → GPS: (38.98590252777778, -107.88789619444445)
DJI_0010.JPG → GPS: (38.98562641666667, -107.88789555555556)
DJI_0011.JPG → GPS: (38.985454861111116, -107.88789669444445)
DJI_0012.JPG → GPS: (38.98519822222222, -107.88789505555556)
DJI_0013.JPG → GPS: (38.98493886111111, -107.88789719444445)
DJI_0014.JPG → GPS: (38.984765361111116, -107.88789927777779)
DJI_0015.JPG → GPS: (38.98450808333333, -107.88790113888889)
DJI_0016.JPG → GPS: (38.984250361111116, -107.88790158333335)
DJI_0017.JPG → GPS: (38.9840795, -107.

In [ ]:
import os

# === Helper: Convert YOLO txt to pixel bounding boxes ===
def load_yolo_boxes(image_dir, image_size=(640, 640)):
    detections = {}
    for fname in os.listdir(image_dir):
        if fname.lower().endswith(".txt"):
            img_name = fname.replace(".txt", ".JPG")
            full_path = os.path.join(image_dir, fname)

            with open(full_path, 'r') as f:
                boxes = []
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    class_id, x, y, w, h = map(float, parts[:5])
                    conf = float(parts[5]) if len(parts) == 6 else 1.0

                    # Convert YOLO format to pixel coords
                    xc, yc, bw, bh = x * image_size[0], y * image_size[1], w * image_size[0], h * image_size[1]
                    x1 = xc - bw / 2
                    y1 = yc - bh / 2
                    x2 = xc + bw / 2
                    y2 = yc + bh / 2

                    boxes.append([x1, y1, x2, y2, conf])
                detections[img_name] = boxes
    return detections

# === Helper: Calculate IoU ===
def calculate_iou(box1, box2):
    # Get coordinates of the intersection rectangle
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Calculate area of intersection
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

    # Calculate area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# === Deduplication Function ===
def deduplicate_detections(detections, overlaps, iou_threshold=0.5):
    cleaned_detections = detections.copy()
    for img1, img2, _ in overlaps:
        if img1 in cleaned_detections and img2 in cleaned_detections:
            boxes1 = cleaned_detections[img1]
            boxes2 = cleaned_detections[img2]

            # Create a list of boxes to remove from img2
            boxes_to_remove = []
            for box2 in boxes2:
                for box1 in boxes1:
                    if calculate_iou(box1, box2) > iou_threshold:
                        boxes_to_remove.append(box2)
                        break # Move to the next box in img2

            # Remove the identified duplicate boxes from img2
            cleaned_detections[img2] = [b for b in boxes2 if b not in boxes_to_remove]

    return cleaned_detections

# ===  usage ===
image_dir = "/content/drive/MyDrive/inference_output/run/labels"
detections = load_yolo_boxes(image_dir)

#  overlaps list (normally from GPS overlap logic)
overlaps = [('DJI_0001.JPG', 'DJI_0002.JPG', 0.361), ('DJI_0002.JPG', 'DJI_0003.JPG', 0.361)]

# Run deduplication
cleaned_detections = deduplicate_detections(detections, overlaps)

print("\n=== Cleaned Detections ===")
for img, boxes in cleaned_detections.items():
    print(f"{img}:")
    for box in boxes:
        print(f"  {box}")

In [37]:
!pip install opencv-python-headless

In [36]:
import os
import cv2

labels_folder = "/content/drive/MyDrive/inference_output/run/labels"
images_folder = "/content/drive/MyDrive/Gregg1_2"

def build_global_tree_list(deduplicated, labels_folder, images_folder):
    global_tree_list = []

    for img_name, box_index in deduplicated:
        img_path = os.path.join(images_folder, img_name)
        label_path = os.path.join(labels_folder, os.path.splitext(img_name)[0] + ".txt")

        if not os.path.exists(img_path) or not os.path.exists(label_path):
            continue

        image = cv2.imread(img_path)
        h, w = image.shape[:2]

        with open(label_path, 'r') as f:
            lines = f.readlines()

        if box_index < len(lines):
            yolo_box = lines[box_index].strip()
            bbox = yolo_to_bbox(yolo_box, w, h)

            x_center = (bbox[0] + bbox[2]) // 2
            y_center = (bbox[1] + bbox[3]) // 2

            tree_data = {
                "image": img_name,
                "bbox": bbox,
                "center": (x_center, y_center)
            }
            global_tree_list.append(tree_data)

    return global_tree_list

In [24]:
import os
import cv2
import math

# === Paths ===
labels_folder = "/content/drive/MyDrive/inference_output/run/labels"
images_folder = "/content/drive/MyDrive/Gregg1_2"

frame1 = "DJI_0170.JPG"
frame2 = "DJI_0171.JPG"

distance_thresh = 50  # pixels

# === YOLO to pixel bounding box ===
def yolo_to_bbox(yolo_str, img_w, img_h):
    cls, x_c, y_c, w, h = map(float, yolo_str.strip().split())
    x1 = int((x_c - w / 2) * img_w)
    y1 = int((y_c - h / 2) * img_h)
    x2 = int((x_c + w / 2) * img_w)
    y2 = int((y_c + h / 2) * img_h)
    return [x1, y1, x2, y2]

# === Center point distance ===
def center_distance(c1, c2):
    return math.sqrt((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2)

# === Load detections from a frame ===
def load_detections(frame):
    image_path = os.path.join(images_folder, frame)
    label_path = os.path.join(labels_folder, frame.replace(".JPG", ".txt"))

    if not os.path.exists(image_path) or not os.path.exists(label_path):
        print(f"Missing image or label for {frame}")
        return []

    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    with open(label_path, 'r') as f:
        lines = f.readlines()

    detections = []
    for line in lines:
        bbox = yolo_to_bbox(line, w, h)
        x_center = (bbox[0] + bbox[2]) // 2
        y_center = (bbox[1] + bbox[3]) // 2
        detections.append({
            "frame": frame,
            "center": (x_center, y_center),
            "bbox": bbox
        })
    print(f"Loaded {len(detections)} detections from {frame}")
    return detections

# === Deduplicate between two frames ===
def deduplicate_frames(dets1, dets2):
    unique = dets1.copy()  # Start with all objects from frame1

    for det in dets2:
        duplicate = False
        for ref in dets1:  # compare only with frame1 objects
            if center_distance(det["center"], ref["center"]) < distance_thresh:
                duplicate = True
                break
        if not duplicate:
            unique.append(det)

    print(f"🧮 Frame 1 Detections: {len(dets1)}")
    print(f"🧮 Frame 2 Detections: {len(dets2)}")
    print(f"✅ Unique Objects after Deduplication: {len(unique)}")

    return unique

# === Run everything ===
dets_frame1 = load_detections(frame1)
dets_frame2 = load_detections(frame2)
unique_objects = deduplicate_frames(dets_frame1, dets_frame2)

# Optional: print all unique detections
#for obj in unique_objects:
    #print(obj)


Loaded 156 detections from DJI_0170.JPG
Loaded 266 detections from DJI_0171.JPG
🧮 Frame 1 Detections: 156
🧮 Frame 2 Detections: 266
✅ Unique Objects after Deduplication: 336


In [28]:
import os
import cv2
import math

# Paths
labels_folder = "/content/drive/MyDrive/inference_output/run/labels"
images_folder = "/content/drive/MyDrive/Gregg1_2"

# --- Helper: Convert YOLO to pixel bounding box ---
def yolo_to_bbox(yolo_str, img_w, img_h):
    cls, x_c, y_c, w, h = map(float, yolo_str.strip().split())
    x1 = int((x_c - w/2) * img_w)
    y1 = int((y_c - h/2) * img_h)
    x2 = int((x_c + w/2) * img_w)
    y2 = int((y_c + h/2) * img_h)
    return [x1, y1, x2, y2]

# --- Distance between two box centers ---
def center_distance(c1, c2):
    return math.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

# --- Main: Count and Deduplicate ---
def count_and_deduplicate(distance_thresh=50):
    all_detections = []

    print("🚀 Starting tree detection processing...\n")

    for label_file in os.listdir(labels_folder):
        if not label_file.endswith(".txt"):
            continue

        image_name = label_file.replace(".txt", ".JPG")
        label_path = os.path.join(labels_folder, label_file)
        image_path = os.path.join(images_folder, image_name)

        if not os.path.exists(image_path):
            print(f"❌ Image not found: {image_path}")
            continue

        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        with open(label_path, 'r') as f:
            lines = f.readlines()

        #print(f"📂 Loaded {len(lines)} detections from {image_name}")

        for line in lines:
            bbox = yolo_to_bbox(line, w, h)
            x_center = (bbox[0] + bbox[2]) // 2
            y_center = (bbox[1] + bbox[3]) // 2
            all_detections.append({
                "frame": image_name,
                "center": (x_center, y_center),
                "bbox": bbox
            })

    print(f"\n🧮 Total Detections Before Deduplication: {len(all_detections)}")

    # Deduplication logic
    unique = []
    for det in all_detections:
        duplicate = False
        for uniq in unique:
            if center_distance(det["center"], uniq["center"]) < distance_thresh:
                duplicate = True
                break
        if not duplicate:
            unique.append(det)

    print(f"✅ Total Unique Trees After Deduplication: {len(unique)}")

    return {
        "detected_count": len(all_detections),
        "unique_count": len(unique),
        "unique_trees": unique
    }

if __name__ == "__main__":
    result = count_and_deduplicate(distance_thresh=50)

    print("\n📊 Final Summary:")
    print(f"Total Detected Trees: {result['detected_count']}")
    print(f"Total Unique Trees : {result['unique_count']}")

🚀 Starting tree detection processing...


🧮 Total Detections Before Deduplication: 50108
✅ Total Unique Trees After Deduplication: 2830

📊 Final Summary:
Total Detected Trees: 50108
Total Unique Trees : 2830


In [18]:
def load_all_boxes(images_folder, labels_folder):
    """
    Load all bounding box detections from .txt files in the labels folder.
    """
    all_detections = {}
    for label_file in os.listdir(labels_folder):
        if label_file.endswith('.txt'):
            frame_name = label_file.replace(".txt", ".JPG")
            detections = load_detections(frame_name)
            if detections:
                all_detections[frame_name] = detections
    return all_detections